## Scaling to 10,000 Samples with Balanced Data

- **Purpose:** Improve performance by doubling the dataset and ensuring equal class representation.
- **Details:** Custom `BalancedStreamingDataset` ensures **~625** samples per class, lifting accuracy to **77.2%**.
- **Outcome:** Balanced data significantly boosts model fairness and financial class accuracy.


In [ ]:
# Step 1: Installing required libraries & Setting up the Environment
!pip install -q transformers datasets torch torchvision accelerate

import torch
from transformers import ViTImageProcessor, ViTForImageClassification, TrainingArguments, Trainer
from datasets import load_dataset, IterableDataset
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from collections import defaultdict

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
print(f"GPU name: {torch.cuda.get_device_name(0)}")
!free -h
!df -h

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 32.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 25.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 38.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 64.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
# Step 2: Load Dataset with Balanced Streaming
dataset = load_dataset("aharley/rvl_cdip", streaming=True)
label_map = {0: "letter", 1: "form", 2: "email", 3: "handwritten", 4: "advertisement",
             5: "scientific report", 6: "scientific publication", 7: "specification",
             8: "file folder", 9: "news article", 10: "budget", 11: "invoice",
             12: "presentation", 13: "questionnaire", 14: "resume", 15: "memo"}
num_labels = len(label_map)
processor = ViTImageProcessor.from_pretrained("google/vit-base-patch16-224-in21k")

class BalancedStreamingDataset(IterableDataset):
    def __init__(self, dataset_split, total_samples, num_classes=16):
        self.dataset = dataset_split
        self.total_samples = total_samples
        self.target_per_class = total_samples // num_classes
        self.num_classes = num_classes
        self._epoch = 0

    def __iter__(self):
        class_counts = defaultdict(int)
        samples_yielded = 0
        for example in self.dataset:
            label = example["label"]
            if class_counts[label] < self.target_per_class:
                class_counts[label] += 1
                image = example["image"].convert("RGB")
                inputs = processor(images=image, return_tensors="pt")
                yield {
                    "pixel_values": inputs["pixel_values"].squeeze(0),
                    "labels": label
                }
                samples_yielded += 1
                if samples_yielded >= self.total_samples:
                    break

    def __len__(self):
        return self.total_samples

    def set_epoch(self, epoch: int):
        self._epoch = epoch

train_size = 10000
val_size = 2000
test_size = 2000
train_dataset = BalancedStreamingDataset(dataset["train"], train_size)
val_dataset = BalancedStreamingDataset(dataset["validation"], val_size)
test_dataset = BalancedStreamingDataset(dataset["test"], test_size)
print(f"Training size: {train_size}, Validation size: {val_size}, Test size: {test_size}")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/6.15k [00:00<?, ?B/s]

rvl_cdip.py:   0%|          | 0.00/4.80k [00:00<?, ?B/s]

dataset_infos.json:   0%|          | 0.00/2.64k [00:00<?, ?B/s]

The repository for aharley/rvl_cdip contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/aharley/rvl_cdip.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


preprocessor_config.json:   0%|          | 0.00/160 [00:00<?, ?B/s]

Training size: 10000, Validation size: 2000, Test size: 2000


In [ ]:
# Step 3: Load Pre-trained ViT Model
model = ViTForImageClassification.from_pretrained(
    "google/vit-base-patch16-224-in21k",
    num_labels=num_labels,
    ignore_mismatched_sizes=True
)
model.to(device)
print(f"GPU memory allocated: {torch.cuda.memory_allocated(device) / 1024**2:.2f} MB")

config.json:   0%|          | 0.00/502 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


GPU memory allocated: 327.34 MB


In [ ]:
# Step 4: Define Metrics
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    accuracy = accuracy_score(labels, predictions)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average="weighted")
    return {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1": f1
    }

In [ ]:
# Step 5: Set Up Training Arguments (Corrected)
training_args = TrainingArguments(
    output_dir="./rvl_cdip_vit",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=5,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    logging_dir="./logs",
    logging_steps=50,
    fp16=True,
    gradient_accumulation_steps=8,
)

In [ ]:
# Step 6: Train the Model with Early Stopping
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]  # Stop if no improvement for 2 epochs
)
print("Starting training...")
trainer.train()
print(f"GPU memory allocated post-training: {torch.cuda.memory_allocated(device) / 1024**2:.2f} MB")

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Starting training...


wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: gavhaneprasad14092001 (gavhaneprasad14092001-indian-school-of-mines) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,1.345800,1.364739,0.627500,0.659117,0.627500,0.617502
2,0.943600,1.083000,0.695500,0.722146,0.695500,0.697258
3,0.691400,1.037812,0.697000,0.749832,0.697000,0.705176
4,0.382800,0.834844,0.770500,0.777305,0.770500,0.772086


GPU memory allocated post-training: 1000.63 MB


In [ ]:
# Step 7: Evaluate on Test Set
test_dataset = BalancedStreamingDataset(dataset["test"], test_size)
test_results = trainer.evaluate(test_dataset)
print("Test Results:", test_results)

predictions = trainer.predict(test_dataset)
preds = np.argmax(predictions.predictions, axis=-1)
labels = predictions.label_ids
financial_classes = [1, 10, 11, 15]
for cls in financial_classes:
    mask = labels == cls
    cls_preds = preds[mask]
    cls_labels = labels[mask]
    acc = accuracy_score(cls_labels, cls_preds) if len(cls_labels) > 0 else 0
    print(f"Accuracy for {label_map[cls]} (label {cls}): {acc:.4f}")

Test Results: {'eval_loss': 0.8585492968559265, 'eval_accuracy': 0.772, 'eval_precision': 0.7773170888759323, 'eval_recall': 0.772, 'eval_f1': 0.7734974692777712, 'eval_runtime': 175.917, 'eval_samples_per_second': 11.369, 'eval_steps_per_second': 2.842, 'epoch': 4.9856}
Accuracy for form (label 1): 0.6480
Accuracy for budget (label 10): 0.6720
Accuracy for invoice (label 11): 0.6800
Accuracy for memo (label 15): 0.7280


In [ ]:
# Step 8: Save the Model
model.save_pretrained("./rvl_cdip_vit_model")
processor.save_pretrained("./rvl_cdip_vit_model")
!du -sh ./rvl_cdip_vit_model
!df -h

328M	./rvl_cdip_vit_model
Filesystem      Size  Used Avail Use% Mounted on
overlay         113G   45G   69G  40% /
tmpfs            64M     0   64M   0% /dev
shm             5.7G   16K  5.7G   1% /dev/shm
/dev/root       2.0G  1.2G  820M  59% /usr/sbin/docker-init
tmpfs           6.4G  288K  6.4G   1% /var/colab
/dev/sda1        92G   71G   22G  77% /kaggle/input
tmpfs           6.4G     0  6.4G   0% /proc/acpi
tmpfs           6.4G     0  6.4G   0% /proc/scsi
tmpfs           6.4G     0  6.4G   0% /sys/firmware
